In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split , KFold, cross_val_score
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.metrics import classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import DBSCAN
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
sDbTeste = "test.csv"
sDbTrain = "train.csv"

In [3]:
dbTrain = pd.read_csv(sDbTrain, low_memory=False)
dbTest = pd.read_csv(sDbTeste, low_memory=False)

In [4]:
dbTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227122 entries, 0 to 227121
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 227122 non-null  int64  
 1   NR_SEQ_REQUISICAO          227122 non-null  int64  
 2   NR_SEQ_ITEM                227122 non-null  int64  
 3   DT_REQUISICAO              227122 non-null  int64  
 4   DS_TIPO_GUIA               227122 non-null  object 
 5   DT_NASCIMENTO              227112 non-null  float64
 6   NR_PRODUTO                 227122 non-null  int64  
 7   DS_TIPO_PREST_SOLICITANTE  227122 non-null  object 
 8   DS_CBO                     227122 non-null  object 
 9   DS_TIPO_CONSULTA           10511 non-null   object 
 10  QT_TEMPO_DOENCA            266 non-null     float64
 11  DS_UNIDADE_TEMPO_DOENCA    266 non-null     object 
 12  DS_TIPO_DOENCA             531 non-null     object 
 13  DS_INDICACAO_ACIDENTE      20

In [6]:
dbTrain = dbTrain.drop(['DS_TIPO_SAIDA'], axis = 1)

In [7]:
def preproc(df):
  #NUMERIC
    df.QT_DIA_SOLICITADO = df.QT_DIA_SOLICITADO.fillna(0)
    df.QT_TEMPO_DOENCA = df.QT_TEMPO_DOENCA.fillna(0)
    df.CD_GUIA_REFERENCIA = df.CD_GUIA_REFERENCIA.fillna(0)
    df.NR_SEQ_REQUISICAO = df.NR_SEQ_REQUISICAO.fillna(0)
    df.CD_GUIA_REFERENCIA = df.CD_GUIA_REFERENCIA.fillna(0)
    df.CD_ITEM = df.CD_ITEM.fillna(0)
    #CATEGORIC
    df.DS_INDICACAO_ACIDENTE = df.DS_INDICACAO_ACIDENTE.fillna('0')
    df.DS_TIPO_INTERNACAO = df.DS_TIPO_INTERNACAO.fillna('0')
    df.DS_TIPO_ATENDIMENTO = df.DS_TIPO_ATENDIMENTO.fillna('0')
    df.DS_UNIDADE_TEMPO_DOENCA = df.DS_UNIDADE_TEMPO_DOENCA.fillna('0')
    df.DS_TIPO_CONSULTA = df.DS_TIPO_CONSULTA.fillna('0')
    df.DS_TIPO_DOENCA = df.DS_TIPO_DOENCA.fillna('0')
    df.DS_REGIME_INTERNACAO = df.DS_REGIME_INTERNACAO.fillna('0')
    df.DS_TIPO_ACOMODACAO = df.DS_TIPO_ACOMODACAO.fillna('0')
    df.DS_INDICACAO_CLINICA = df.DS_INDICACAO_CLINICA.fillna('0')
    df.CD_CID = df.CD_CID.fillna('0')
    return df
  

In [8]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np

def standardScalerFunc(data):
  ss = StandardScaler()
  sstransformed = ss.fit_transform(data)

  return ss, pd.DataFrame(sstransformed)

def labelEncoderFunc(data):
  le = LabelEncoder()
  sstransformed = ss.fit_transform(data) 

  return le, pd.DataFrame(sstransformed)

def oneHotEncoderFunc(data):
  ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
  ohetransformed = ohe.fit_transform(data)
  
  return ohe, pd.DataFrame(ohetransformed)

In [9]:
dbTrain = preproc(dbTrain)

In [10]:

numeric = dbTrain[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']]
categoric = dbTrain[['DS_TIPO_PREST_SOLICITANTE','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 
                     'DS_INDICACAO_ACIDENTE', 'DS_TIPO_GUIA', 'DS_GRUPO', 'DS_CBO', 'DS_TIPO_CONSULTA', 'DS_TIPO_DOENCA', 'DS_REGIME_INTERNACAO', 'DS_TIPO_ACOMODACAO', 'DS_TIPO_ITEM']]
target = dbTrain['DS_STATUS_ITEM']

In [11]:
x = pd.concat([numeric, categoric], axis=1)
print(len(x))
x = x.dropna()
print(len(x))
y = target

227122
227122


In [12]:
#split Train e Test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0) # 80% training 20% test

ss, x_train_ss = standardScalerFunc(x_train[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
ohe, x_train_ohe = oneHotEncoderFunc(x_train[['DS_TIPO_PREST_SOLICITANTE','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_GUIA', 'DS_GRUPO', 'DS_CBO', 'DS_TIPO_CONSULTA', 'DS_TIPO_DOENCA', 'DS_REGIME_INTERNACAO', 'DS_TIPO_ACOMODACAO', 'DS_TIPO_ITEM']])

x_train = pd.concat([x_train_ss, x_train_ohe], axis=1)

x_test_ss = ss.transform(x_test[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
x_test_ohe = ohe.transform(x_test[['DS_TIPO_PREST_SOLICITANTE','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_GUIA', 'DS_GRUPO', 'DS_CBO', 'DS_TIPO_CONSULTA', 'DS_TIPO_DOENCA', 'DS_REGIME_INTERNACAO', 'DS_TIPO_ACOMODACAO', 'DS_TIPO_ITEM']])

x_test = pd.concat([pd.DataFrame(x_test_ss), pd.DataFrame(x_test_ohe)], axis=1)

In [13]:
dbTest_pp = preproc(dbTest)
test_ss = ss.transform(dbTest_pp [['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
test_ohe = ohe.transform(dbTest_pp[['DS_TIPO_PREST_SOLICITANTE','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 
                                    'DS_TIPO_GUIA', 'DS_GRUPO', 'DS_CBO', 'DS_TIPO_CONSULTA', 'DS_TIPO_DOENCA', 'DS_REGIME_INTERNACAO', 'DS_TIPO_ACOMODACAO', 'DS_TIPO_ITEM']])

test = pd.concat([pd.DataFrame(test_ss), pd.DataFrame(test_ohe)], axis=1)

In [14]:
y_train = pd.get_dummies(y_train)

In [15]:
#Classifier
import keras
from keras.utils import np_utils
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers

def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=142, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dropout(dropout), )
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='Nadam',metrics=["accuracy"])
    return model


In [16]:
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.layers import Dense, Input, Dropout
from keras import Sequential

keras_estimator = KerasClassifier(build_fn=create_model, verbose=1)

In [17]:
keras_estimator.fit(x_train, y_train, epochs = 10, batch_size = 256)

C:\Users\ggare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/15
710/710 [==============================] - 2s 2ms/step - loss: 0.2033 - accuracy: 0.6948
Epoch 2/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1978 - accuracy: 0.7030
Epoch 3/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1958 - accuracy: 0.7070
Epoch 4/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1946 - accuracy: 0.7095
Epoch 5/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1935 - accuracy: 0.7107
Epoch 6/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1929 - accuracy: 0.7120
Epoch 7/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1922 - accuracy: 0.7131
Epoch 8/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1917 - accuracy: 0.7137
Epoch 9/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1912 - accuracy: 0.7150
Epoch 10/15
710/710 [==============================] - 1s 2ms/step - loss: 0.1909 - accuracy: 0.7150

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x000002A9BEB2C160>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [18]:
y_pred = keras_estimator.predict(x_test)

1420/1420 [==============================] - 1s 624us/step


In [19]:
y_pred = np.argmax(y_pred, axis = 1)

In [20]:
report_pred = pd.DataFrame(y_pred).replace('Autorizado', 0).replace('Negado', 1)

In [21]:
report_test = pd.DataFrame(y_test).replace('Autorizado', 0).replace('Negado', 1)

In [22]:
print("Accuracy:",metrics.accuracy_score(report_test['DS_STATUS_ITEM'], report_pred[0]))
print("Precision:",metrics.precision_score(report_test['DS_STATUS_ITEM'], report_pred[0]))
print("Recall:",metrics.recall_score(report_test['DS_STATUS_ITEM'], report_pred[0]))
print("F1:",metrics.f1_score(report_test['DS_STATUS_ITEM'], report_pred[0]))

Accuracy: 0.713241607044579
Precision: 0.6176956587051224
Recall: 0.28177893510587265
F1: 0.3870117647058823


In [23]:
print(classification_report(report_test['DS_STATUS_ITEM'], report_pred[0]))

              precision    recall  f1-score   support

           0       0.73      0.92      0.81     30832
           1       0.62      0.28      0.39     14593

    accuracy                           0.71     45425
   macro avg       0.67      0.60      0.60     45425
weighted avg       0.69      0.71      0.68     45425



In [31]:
y_pred = keras_estimator.predict(test)

5817/5817 [==============================] - 4s 650us/step


In [32]:
y_pred = np.argmax(y_pred, axis = 1)

In [33]:
report_pred = pd.DataFrame(y_pred).replace('Autorizado', 0).replace('Negado', 1)

In [34]:
y_frame = pd.DataFrame(report_pred)

In [35]:
y_final = y_frame.replace(0,'Autorizado').replace(1, 'Negado')

In [38]:
from itertools import zip_longest
submission = pd.DataFrame.from_records(zip_longest(dbTest['Unnamed: 0'], y_final.values.flatten()), columns=['ID', 'DS_STATUS_ITEM'])

submission.set_index('ID').to_csv('submission Final.csv')

In [39]:
pd.read_csv('submission Final.csv')

,ID,DS_STATUS_ITEM
0,0,Negado
1,5,Autorizado
2,8,Autorizado
3,13,Autorizado
4,18,Autorizado
...,...,...
186139,413210,Autorizado
186140,413218,Autorizado
186141,413221,Negado
186142,413248,Negado
